# Predicting gentrification in Chicago
*Midpoint Report by The Binary Beasts*

In this preliminary report, we concentrated on collecting data from the Census Bureau and running a model that could predict gentrification based on basic sociodemographic variables. For the next deriverable, we expect to incorporate more variables related to the actual link between the presence of environmental resources, such as
park and garden facilities, natural features, and air pollution, and increased susceptibility to  gentrification. As discussed in our literature review, our exact research questions will depend on the specifics of our data, namely if we can locate data corresponding to the construction or enhancement of parks, etc. In that case, we would like to draw out the effect of new environmental access on gentrification, returning to the idea of “green gentrification” discussed earlier. If we can only find static data on parks, we can still look at how their presence impacts the model of gentrification risk by comparing socioeconomic change over time in areas with or without these resources.

### Data Collection

For this report we defined our outcome variable as...

In the same way, we integrated as possible features...

### Exploring the dataset

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import requests
import itertools
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
df = pd.read_csv()

**Outcome variable**

**Features**

### Logistic regression